## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тренировочное множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

lines = list(open('./news_train.txt', 'r', encoding='utf-8'))

In [15]:
import pandas as pd

from pymorphy2.tokenizers import simple_word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest

from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

from tqdm import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings('ignore')

## 0. Читаем данные

In [2]:
df_train = pd.read_csv('../data/news_train.txt', sep='\t', names=['class', 'title', 'text'])
df_test = pd.read_csv('../data/news_test.txt', sep='\t', names=['class', 'title', 'text'])
df_train.head()

,class,title,text
0,sport,Овечкин пожертвовал детской хоккейной школе ав...,Нападающий «Вашингтон Кэпиталз» Александр Овеч...
1,culture,Рекордно дорогую статую майя признали подделкой,"Власти Мексики объявили подделкой статую майя,..."
2,science,Samsung представила флагман в защищенном корпусе,Южнокорейская Samsung анонсировала защищенную ...
3,sport,С футболиста «Спартака» сняли четырехматчевую ...,Контрольно-дисциплинарный комитет (КДК) РФС сн...
4,media,Hopes & Fears объединится с The Village,Интернет-издание Hopes & Fears объявило о свое...


## 1. Токенизация и стемминг

In [9]:
def tokenize(line):
    tokens = simple_word_tokenize(line)
    stems = [
        snowball.stem(x.lower())
        for x in tokens
        if x not in stopwords_ru
    ]
    return stems

In [10]:
snowball = SnowballStemmer(language="russian")
stopwords_ru = stopwords.words('russian')

df_train['text_stems'] = df_train.text.progress_apply(tokenize)
df_test['text_stems'] = df_test.text.progress_apply(tokenize)

100%|█████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:22<00:00, 135.96it/s]


In [11]:
df_train.head()

,class,title,text,text_stems
0,sport,Овечкин пожертвовал детской хоккейной школе ав...,Нападающий «Вашингтон Кэпиталз» Александр Овеч...,"[напада, «, вашингтон, кэпиталз, », александр,..."
1,culture,Рекордно дорогую статую майя признали подделкой,"Власти Мексики объявили подделкой статую майя,...","[власт, мексик, объяв, подделк, стат, май, ,, ..."
2,science,Samsung представила флагман в защищенном корпусе,Южнокорейская Samsung анонсировала защищенную ...,"[южнокорейск, samsung, анонсирова, защищен, ве..."
3,sport,С футболиста «Спартака» сняли четырехматчевую ...,Контрольно-дисциплинарный комитет (КДК) РФС сн...,"[контрольно-дисциплинарн, комитет, (, кдк, ), ..."
4,media,Hopes & Fears объединится с The Village,Интернет-издание Hopes & Fears объявило о свое...,"[интернет-издан, hopes, &, fears, объяв, сво, ..."


## 2. Word2Vec

In [12]:
word2vec = Word2Vec(df_train.text_stems,
                    size=128,
                    window=5,
                    sg=1,  # skip-gram
                    workers=2)

vec = word2vec.wv.vectors

In [13]:
word2vec.most_similar('вашингтон')

[('нью-джерс', 0.7245336771011353),
 ('кэпиталс', 0.7167192697525024),
 ('колорад', 0.7156737446784973),
 ('миннесот', 0.7052792310714722),
 ('эвеланш', 0.6969218254089355),
 ('сан-хос', 0.6897116899490356),
 ('пантерс', 0.677248477935791),
 ('лос-анджелес', 0.674209475517273),
 ('рейнджерс', 0.6714686155319214),
 ('оклахом', 0.6680904626846313)]

In [14]:
word2vec.most_similar('питон')

[('жилищ', 0.9093278646469116),
 ('побежа', 0.9044625759124756),
 ('медвежат', 0.9021425247192383),
 ('копытн', 0.9020285606384277),
 ('заперт', 0.9017809629440308),
 ('колодец', 0.8995034694671631),
 ('крокодил', 0.8992606401443481),
 ('медведиц', 0.8989448547363281),
 ('загон', 0.8969249725341797),
 ('поеда', 0.8950693011283875)]

## 2. Классификатор на tf–idf

In [28]:
def evaluate(pipeline, X_train, y_train, X_val, y_val):
    y_val_pred = pipeline.predict(X_val)
    print(f'{pipeline.steps[-1][1].__class__.__name__} test accuracy: {accuracy_score(y_val, y_val_pred)}')

In [17]:
tfidf = TfidfVectorizer()

X_train = tfidf.fit_transform(df_train.text_stems.apply(lambda x: ' '.join(x)))
y_train = df_train['class'].values

X_test = tfidf.transform(df_test.text_stems.apply(lambda x: ' '.join(x)))
y_test = df_test['class'].values

In [32]:
models = {
    DummyClassifier(): {},
    LogisticRegression(class_weight='balanced'): {'model__solver': ['newton-cg', 'lbfgs', 'saga']},
    SVC(class_weight='balanced'): {'model__kernel': ['linear', 'rbf']},
}

for model, params in models.items():
    pipeline = Pipeline([
        ('feature_selection', SelectKBest()),
        ('model', model),
    ])

    param_grid = {
        **params,
        **{'feature_selection__k': [6000, 8000]}
    }

    gs = GridSearchCV(pipeline,
                      param_grid=param_grid,
                      cv=3,
                      refit=True,
                      verbose=0)

    gs.fit(X_train, y_train)
    evaluate(gs.best_estimator_, X_train, y_train, X_test, y_test)

DummyClassifier test accuracy: 0.12833333333333333
LogisticRegression test accuracy: 0.8733333333333333
SVC test accuracy: 0.8726666666666667
